In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
import catboost as ctb
from sklearn.model_selection import KFold

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier

c:\Users\lenovo\anaconda3\lib\site-packages\dask\dataframe\utils.py:366: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
c:\Users\lenovo\anaconda3\lib\site-packages\dask\dataframe\utils.py:366: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
c:\Users\lenovo\anaconda3\lib\site-packages\dask\dataframe\utils.py:366: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


In [19]:
train_data_X =pd.read_csv('train.csv')
test_data = pd.read_csv("test.csv")

In [8]:
train_data_X=train_data_X.fillna(-99)
test_data=test_data.fillna(-99)

In [20]:
train_data_y = train_data_X['label']
train_data_X = train_data_X.drop(['cust_id','label'],axis=1)

In [15]:
oof_lgb = np.zeros(train_data_X.shape[0])
oof_xgb = np.zeros(train_data_X.shape[0])
oof_cat = np.zeros(train_data_X.shape[0])
oof_rft = np.zeros(train_data_X.shape[0])
oof_ext = np.zeros(train_data_X.shape[0])

test_output_df = pd.DataFrame(columns=['lgb','xgb','cat'],index=range(test_data.shape[0]))
test_output_df = test_output_df.fillna(0)

In [21]:
kfold = KFold(random_state=800,shuffle=True)
lgb_reg = lgb.LGBMRegressor(max_depth=8,
                           n_estimators=3000,
                           subsample=0.8,
                           colsample_bytree=0.8,
                           random_state=32)

cat_reg = ctb.CatBoostRegressor(learning_rate=0.1,
                               depth=8,
                               random_seed=32,
                               )

xgb_reg = xgb.XGBRegressor(max_depth=8,
                          learning_rate=0.1,
                          n_estimators=3000,
                          n_jobs=4,
                          colsample_bytree=0.8,
                          subsample=0.8,
                          random_state=32,
                          )
from sklearn.ensemble import RandomForestRegressor,ExtraTreesRegressor,GradientBoostingRegressor

ext_reg =  ExtraTreesRegressor(max_depth=8,
                        min_samples_split=100,
                          min_samples_leaf=20,
                          random_state=32,)
rft_reg=RandomForestRegressor(max_depth=8,
                        min_samples_split=100,
                          min_samples_leaf=20,
                          random_state=32,)

for train_idx, valid_idx in kfold.split(train_data_X):
    train_x = train_data_X.loc[train_idx]
    train_y = train_data_y.loc[train_idx]
    valid_x = train_data_X.loc[valid_idx]
    valid_y = train_data_y.loc[valid_idx]
    lgb_reg.fit(train_x,train_y, eval_set=[(train_x,train_y),(valid_x,valid_y)],early_stopping_rounds=30)
    xgb_reg.fit(train_x,train_y, eval_set=[(train_x,train_y),(valid_x,valid_y)],early_stopping_rounds=30)
    cat_reg.fit(train_x,train_y, eval_set=[(train_x,train_y),(valid_x,valid_y)],early_stopping_rounds=30)
    oof_lgb[valid_idx] = lgb_reg.predict(valid_x)
    oof_xgb[valid_idx] = xgb_reg.predict(valid_x)
    oof_cat[valid_idx] = cat_reg.predict(valid_x)
    test_output_df['xgb'] += xgb_reg.predict(test_data.drop('cust_id',axis=1))
    test_output_df['lgb'] += lgb_reg.predict(test_data.drop('cust_id',axis=1))
    test_output_df['cat'] += cat_reg.predict(test_data.drop('cust_id',axis=1))

test_output_df['lgb'] = test_output_df['lgb'] / 5
test_output_df['xgb'] = test_output_df['xgb'] / 5
test_output_df['cat'] = test_output_df['cat'] / 5
oof_df = pd.DataFrame({'xgb':oof_xgb,'lgb':oof_lgb,'cat':oof_cat})
from sklearn.linear_model import LogisticRegression

log_r = LogisticRegression()
log_r.fit(oof_df, train_data_y)
final_pre1 = log_r.predict(test_output_df)
from sklearn.linear_model import LinearRegression


lrg = LinearRegression()
lrg.fit(oof_df, train_data_y)
final_pre2 = lrg.predict(test_output_df)
final_pre3 = np.where(final_pre2>=0.5, 1, 0)
#final_pre2.to_csv('x.csv')

c:\Users\lenovo\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	training's l2: 0.0909487	valid_1's l2: 0.0872042
[2]	training's l2: 0.0897127	valid_1's l2: 0.0865637
[3]	training's l2: 0.0886626	valid_1's l2: 0.0859463
[4]	training's l2: 0.0876685	valid_1's l2: 0.0854842
[5]	training's l2: 0.0867646	valid_1's l2: 0.0850764
[6]	training's l2: 0.0858709	valid_1's l2: 0.0845361
[7]	training's l2: 0.0850081	valid_1's l2: 0.0840826
[8]	training's l2: 0.0841545	valid_1's l2: 0.0836617
[9]	training's l2: 0.0833849	valid_1's l2: 0.0834524
[10]	training's l2: 0.08264	valid_1's l2: 0.0832213
[11]	training's l2: 0.0819611	valid_1's l2: 0.0830288
[12]	training's l2: 0.0812497	valid_1's l2: 0.0828409
[13]	training's l2: 0.0805639	valid_1's l2: 0.0826171
[14]	training's l2: 0.0799509	valid_1's l2: 0.0823416
[15]	training's l2: 0.0793185	valid_1's l2: 0.0821423
[16]	training's l2: 0.0787332	valid_1's l2: 0.0819672
[17]	training's l2: 0.0781773	valid_1's l2: 0.0817972
[18]	training's l2: 0.0776237	valid_1's l2: 0.0816755
[19]	training's l2: 0.0770612	valid_1's

c:\Users\lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[3]	validation_0-rmse:0.38675	validation_1-rmse:0.39468
[4]	validation_0-rmse:0.36743	validation_1-rmse:0.37722
[5]	validation_0-rmse:0.35034	validation_1-rmse:0.36213
[6]	validation_0-rmse:0.33542	validation_1-rmse:0.34914
[7]	validation_0-rmse:0.32184	validation_1-rmse:0.33857
[8]	validation_0-rmse:0.31048	validation_1-rmse:0.32949
[9]	validation_0-rmse:0.30020	validation_1-rmse:0.32223
[10]	validation_0-rmse:0.29173	validation_1-rmse:0.31587
[11]	validation_0-rmse:0.28470	validation_1-rmse:0.31073
[12]	validation_0-rmse:0.27779	validation_1-rmse:0.30617
[13]	validation_0-rmse:0.27151	validation_1-rmse:0.30229
[14]	validation_0-rmse:0.26654	validation_1-rmse:0.29912
[15]	validation_0-rmse:0.26114	validation_1-rmse:0.29708
[16]	validation_0-rmse:0.25649	validation_1-rmse:0.29525
[17]	validation_0-rmse:0.25246	validation_1-rmse:0.29382
[18]	validation_0-rmse:0.24924	validation_1-rmse:0.29232
[19]	validation_0-rmse:0.24580	validation_1-rmse:0.29116
[20]	validation_0-rmse:0.24299	validat

c:\Users\lenovo\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


...

c:\Users\lenovo\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	training's l2: 0.0909248	valid_1's l2: 0.0872013
[2]	training's l2: 0.0897399	valid_1's l2: 0.0864027
[3]	training's l2: 0.088689	valid_1's l2: 0.0859644
[4]	training's l2: 0.0876136	valid_1's l2: 0.0855518
[5]	training's l2: 0.0866825	valid_1's l2: 0.0853784
[6]	training's l2: 0.0857256	valid_1's l2: 0.0850496
[7]	training's l2: 0.0847982	valid_1's l2: 0.0848032
[8]	training's l2: 0.0840069	valid_1's l2: 0.0844105
[9]	training's l2: 0.0832211	valid_1's l2: 0.0840901
[10]	training's l2: 0.0824595	valid_1's l2: 0.0838864
[11]	training's l2: 0.0817037	valid_1's l2: 0.0837258
[12]	training's l2: 0.081058	valid_1's l2: 0.0835595
[13]	training's l2: 0.0803722	valid_1's l2: 0.0834311
[14]	training's l2: 0.0796667	valid_1's l2: 0.0831532
[15]	training's l2: 0.0790066	valid_1's l2: 0.0830005
[16]	training's l2: 0.0783798	valid_1's l2: 0.0828473
[17]	training's l2: 0.0778191	valid_1's l2: 0.0827954
[18]	training's l2: 0.0771962	valid_1's l2: 0.0827428
[19]	training's l2: 0.0766994	valid_1's

c:\Users\lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[3]	validation_0-rmse:0.38654	validation_1-rmse:0.39626
[4]	validation_0-rmse:0.36618	validation_1-rmse:0.37889
[5]	validation_0-rmse:0.34895	validation_1-rmse:0.36388
[6]	validation_0-rmse:0.33335	validation_1-rmse:0.35137
[7]	validation_0-rmse:0.32017	validation_1-rmse:0.34069
[8]	validation_0-rmse:0.30853	validation_1-rmse:0.33177
[9]	validation_0-rmse:0.29866	validation_1-rmse:0.32439
[10]	validation_0-rmse:0.28957	validation_1-rmse:0.31833
[11]	validation_0-rmse:0.28130	validation_1-rmse:0.31331
[12]	validation_0-rmse:0.27403	validation_1-rmse:0.30886
[13]	validation_0-rmse:0.26834	validation_1-rmse:0.30557
[14]	validation_0-rmse:0.26264	validation_1-rmse:0.30274
[15]	validation_0-rmse:0.25845	validation_1-rmse:0.30032
[16]	validation_0-rmse:0.25395	validation_1-rmse:0.29814
[17]	validation_0-rmse:0.25013	validation_1-rmse:0.29645
[18]	validation_0-rmse:0.24666	validation_1-rmse:0.29514
[19]	validation_0-rmse:0.24318	validation_1-rmse:0.29376
[20]	validation_0-rmse:0.24042	validat

c:\Users\lenovo\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
c:\Users\lenovo\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


In [12]:
submission=pd.DataFrame(final_pre2)
submission.to_csv('submission.csv',index=False)